In [0]:
#!pip install torch
#!pip install torchvision
#!pip install numpy
#!pip install matplotlib
#!pip install tqdm
#!pip install tensorboardX
#!pip install pretrainedmodels
#%load_ext autoreload
#%autoreload 2
#%matplotlib inline
import torchvision
import torch
import numpy as np
import os
from torch.backends import cudnn
import random
from random import shuffle
import glob
random.seed( 30 )
import pprint
import torch.nn as nn
import matplotlib.pyplot as plt
from torch.utils import data
from torchvision import transforms as T
from PIL import Image
import pdb
import collections
import numpy as np
import time
import datetime
from torch import optim
from torch.autograd import Variable
import torch.nn.functional as F
import csv
from tqdm import tqdm
from collections import defaultdict
import torch.nn as nn
from tensorboardX import SummaryWriter

"""
mp_lock = torch.multiprocessing.RLock()
from torch.multiprocessing import Pool, Process, set_start_method
try:
    set_start_method('spawn')
except RuntimeError:
    pass
"""

writer_train_acc = SummaryWriter('/home/user3/myenv/result/Train_Acc')
writer_valid_acc = SummaryWriter('/home/user3/myenv/result/Valid_Acc')
writer_train_error = SummaryWriter('/home/user3/myenv/result/Train_Loss')
writer_valid_error = SummaryWriter('/home/user3/myenv/result/Valid_Loss')
writer_valid_dice = SummaryWriter('/home/user3/myenv/result/Valid_Dice')
writer_train_dice = SummaryWriter('/home/user3/myenv/result/Train_Dice')
writer_test_acc = SummaryWriter('/home/user3/myenv/result/Test_Acc')

# tensorboard --logdir=/home/nita/anaconda3/Image_Segmentation/result
# tensorboard --logdir=/home/nita/myenv/result
# tensorboard --logdir=/home/colorlab/myenv/result
# tensorboard --logdir=/home/user3/myenv/result


In [0]:
torch.cuda.empty_cache()

In [0]:
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score:
            self.counter += 1
            print("EarlyStopping counter: {:.4f} out of {:.4f}".format(self.counter, self.patience))
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            print("Validation loss decreased: {:.4f} --> {:.4f}".format(self.val_loss_min, val_loss))
        torch.save(model.state_dict(), 'checkpoint.pt')
        self.val_loss_min = val_loss

In [0]:
### Pre-trained RESNet_Encoder

%run resnet.ipynb
import functools
import torch.utils.model_zoo as model_zoo

encoders = {}
encoders.update(resnet_encoders)

def get_encoder(name, encoder_weights=None):
    Encoder = encoders[name]['encoder']
    encoder = Encoder(**encoders[name]['params'])
    encoder.out_shapes = encoders[name]['out_shapes']

    if encoder_weights is not None:
        settings = encoders[name]['pretrained_settings'][encoder_weights]
        encoder.load_state_dict(model_zoo.load_url(settings['url']))

    return encoder

def get_encoder_names():
    return list(encoders.keys())

def get_preprocessing_fn(encoder_name, pretrained='imagenet'):
    settings = encoders[encoder_name]['pretrained_settings']

    if pretrained not in settings.keys():
        raise ValueError('Avaliable pretrained options {}'.format(settings.keys()))

    input_space = settings[pretrained].get('input_space')
    input_range = settings[pretrained].get('input_range')
    mean = settings[pretrained].get('mean')
    std = settings[pretrained].get('std')
    
    return functools.partial(preprocess_input, mean=mean, std=std, input_space=input_space, input_range=input_range)

def preprocess_input(x, mean=None, std=None, input_space='RGB', input_range=None, **kwargs):

    if input_space == 'BGR':
        x = x[..., ::-1].copy()

    if input_range is not None:
        if x.max() > 1 and input_range[1] == 1:
            x = x / 255.

    if mean is not None:
        mean = np.array(mean)
        x = x - mean

    if std is not None:
        std = np.array(std)
        x = x / std

    return x

In [0]:
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence

class conv_block(nn.Module):
    """
    Convolution Block 
    """
    def __init__(self, in_ch, out_ch):
        super(conv_block, self).__init__()
        
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, kernel_size=3, stride=1, padding=1, bias=True),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, kernel_size=3, stride=1, padding=1, bias=True),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True))

    def forward(self, x):

        x = self.conv(x)
        return x
 
class up_conv(nn.Module):
    """
    Up Convolution Block
    """
    def __init__(self, in_ch, out_ch):
        super(up_conv, self).__init__()
        self.up = nn.Sequential(
            nn.Upsample(scale_factor=2),
            nn.Conv2d(in_ch, out_ch, kernel_size=3, stride=1, padding=1, bias=True),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        x = self.up(x)
        return x


class U_Net_noskip(nn.Module):
    """
    UNet - Basic Implementation
    Paper : https://arxiv.org/abs/1505.04597
    """
    def __init__(self, in_ch=3, out_ch=1):
        super(U_Net_noskip, self).__init__()

        n1 = 64
        filters = [n1, n1 * 2, n1 * 4, n1 * 8, n1 * 16]
        
        self.Maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.Maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.Maxpool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.Maxpool4 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.Conv1 = conv_block(in_ch, filters[0])
        self.Conv2 = conv_block(filters[0], filters[1])
        self.Conv3 = conv_block(filters[1], filters[2])
        self.Conv4 = conv_block(filters[2], filters[3])
        self.Conv5 = conv_block(filters[3], filters[4])

        self.Up5 = up_conv(filters[4], filters[3])
        self.Up_conv5 = conv_block(512, filters[3])

        self.Up4 = up_conv(filters[3], filters[2])
        self.Up_conv4 = conv_block(256, filters[2])

        self.Up3 = up_conv(filters[2], filters[1])
        self.Up_conv3 = conv_block(128, filters[1])

        self.Up2 = up_conv(filters[1], filters[0])
        self.Up_conv2 = conv_block(64, filters[0])

        self.Conv = nn.Conv2d(filters[0], out_ch, kernel_size=1, stride=1, padding=0)

        #self.active = torch.nn.Sigmoid()

    def forward(self, x):

        e1 = self.Conv1(x)

        e2 = self.Maxpool1(e1)
        e2 = self.Conv2(e2)

        e3 = self.Maxpool2(e2)
        e3 = self.Conv3(e3)

        e4 = self.Maxpool3(e3)
        e4 = self.Conv4(e4)

        e5 = self.Maxpool4(e4)
        e5 = self.Conv5(e5)

        d5 = self.Up5(e5)
        #print(d5.shape)
        #d5 = torch.cat((e4, d5), dim=1)
        #print(d5.shape)
        d5 = self.Up_conv5(d5)

        d4 = self.Up4(d5)
        #d4 = torch.cat((e3, d4), dim=1)
        d4 = self.Up_conv4(d4)

        d3 = self.Up3(d4)
        #d3 = torch.cat((e2, d3), dim=1)
        d3 = self.Up_conv3(d3)

        d2 = self.Up2(d3)
        #d2 = torch.cat((e1, d2), dim=1)
        d2 = self.Up_conv2(d2)

        out = self.Conv(d2)

        #d1 = self.active(out)

        return out
    
class U_Net(nn.Module):
    """
    UNet - Basic Implementation
    Paper : https://arxiv.org/abs/1505.04597
    """
    def __init__(self, in_ch=3, out_ch=1):
        super(U_Net, self).__init__()

        n1 = 64
        filters = [n1, n1 * 2, n1 * 4, n1 * 8, n1 * 16]
        
        self.Maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.Maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.Maxpool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.Maxpool4 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.Conv1 = conv_block(in_ch, filters[0])
        self.Conv2 = conv_block(filters[0], filters[1])
        self.Conv3 = conv_block(filters[1], filters[2])
        self.Conv4 = conv_block(filters[2], filters[3])
        self.Conv5 = conv_block(filters[3], filters[4])

        self.Up5 = up_conv(filters[4], filters[3])
        self.Up_conv5 = conv_block(filters[4], filters[3])

        self.Up4 = up_conv(filters[3], filters[2])
        self.Up_conv4 = conv_block(filters[3], filters[2])

        self.Up3 = up_conv(filters[2], filters[1])
        self.Up_conv3 = conv_block(filters[2], filters[1])

        self.Up2 = up_conv(filters[1], filters[0])
        self.Up_conv2 = conv_block(filters[1], filters[0])

        self.Conv = nn.Conv2d(filters[0], out_ch, kernel_size=1, stride=1, padding=0)

        #self.active = torch.nn.Sigmoid()

    def forward(self, x):

        e1 = self.Conv1(x)

        e2 = self.Maxpool1(e1)
        e2 = self.Conv2(e2)

        e3 = self.Maxpool2(e2)
        e3 = self.Conv3(e3)

        e4 = self.Maxpool3(e3)
        e4 = self.Conv4(e4)

        e5 = self.Maxpool4(e4)
        e5 = self.Conv5(e5)

        d5 = self.Up5(e5)
        d5 = torch.cat((e4, d5), dim=1)

        d5 = self.Up_conv5(d5)

        d4 = self.Up4(d5)
        d4 = torch.cat((e3, d4), dim=1)
        d4 = self.Up_conv4(d4)

        d3 = self.Up3(d4)
        d3 = torch.cat((e2, d3), dim=1)
        d3 = self.Up_conv3(d3)

        d2 = self.Up2(d3)
        d2 = torch.cat((e1, d2), dim=1)
        d2 = self.Up_conv2(d2)

        out = self.Conv(d2)

        #d1 = self.active(out)

        return out


class Recurrent_block(nn.Module):
    """
    Recurrent Block for R2Unet_CNN
    """
    def __init__(self, out_ch, t=2):
        super(Recurrent_block, self).__init__()

        self.t = t
        self.out_ch = out_ch
        self.conv = nn.Sequential(
            nn.Conv2d(out_ch, out_ch, kernel_size=3, stride=1, padding=1, bias=True),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        for i in range(self.t):
            if i == 0:
                x = self.conv(x)
            out = self.conv(x + x)
        return out


class RRCNN_block(nn.Module):
    """
    Recurrent Residual Convolutional Neural Network Block
    """
    def __init__(self, in_ch, out_ch, t=2):
        super(RRCNN_block, self).__init__()

        self.RCNN = nn.Sequential(
            Recurrent_block(out_ch, t=t),
            Recurrent_block(out_ch, t=t)
        )
        self.Conv = nn.Conv2d(in_ch, out_ch, kernel_size=1, stride=1, padding=0)

    def forward(self, x):
        x1 = self.Conv(x)
        x2 = self.RCNN(x1)
        out = x1 + x2
        return out


class R2U_Net(nn.Module):
    """
    R2U-Unet implementation
    Paper: https://arxiv.org/abs/1802.06955
    """
    def __init__(self, img_ch=3, output_ch=1, t=2):
        super(R2U_Net, self).__init__()

        n1 = 64
        filters = [n1, n1 * 2, n1 * 4, n1 * 8, n1 * 16]

        self.Maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.Maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.Maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.Maxpool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.Upsample = nn.Upsample(scale_factor=2)

        self.RRCNN1 = RRCNN_block(img_ch, filters[0], t=t)

        self.RRCNN2 = RRCNN_block(filters[0], filters[1], t=t)

        self.RRCNN3 = RRCNN_block(filters[1], filters[2], t=t)

        self.RRCNN4 = RRCNN_block(filters[2], filters[3], t=t)

        self.RRCNN5 = RRCNN_block(filters[3], filters[4], t=t)

        self.Up5 = up_conv(filters[4], filters[3])
        self.Up_RRCNN5 = RRCNN_block(filters[4], filters[3], t=t)

        self.Up4 = up_conv(filters[3], filters[2])
        self.Up_RRCNN4 = RRCNN_block(filters[3], filters[2], t=t)

        self.Up3 = up_conv(filters[2], filters[1])
        self.Up_RRCNN3 = RRCNN_block(filters[2], filters[1], t=t)

        self.Up2 = up_conv(filters[1], filters[0])
        self.Up_RRCNN2 = RRCNN_block(filters[1], filters[0], t=t)

        self.Conv = nn.Conv2d(filters[0], output_ch, kernel_size=1, stride=1, padding=0)

        #self.active = torch.nn.Sigmoid()

    def forward(self, x):

        e1 = self.RRCNN1(x)

        e2 = self.Maxpool(e1)
        e2 = self.RRCNN2(e2)

        e3 = self.Maxpool1(e2)
        e3 = self.RRCNN3(e3)

        e4 = self.Maxpool2(e3)
        e4 = self.RRCNN4(e4)

        e5 = self.Maxpool3(e4)
        e5 = self.RRCNN5(e5)

        d5 = self.Up5(e5)
        d5 = torch.cat((e4, d5), dim=1)
        d5 = self.Up_RRCNN5(d5)

        d4 = self.Up4(d5)
        d4 = torch.cat((e3, d4), dim=1)
        d4 = self.Up_RRCNN4(d4)

        d3 = self.Up3(d4)
        d3 = torch.cat((e2, d3), dim=1)
        d3 = self.Up_RRCNN3(d3)

        d2 = self.Up2(d3)
        d2 = torch.cat((e1, d2), dim=1)
        d2 = self.Up_RRCNN2(d2)

        out = self.Conv(d2)

      # out = self.active(out)

        return out

class Attention_block(nn.Module):
    """
    Attention Block
    """

    def __init__(self, F_g, F_l, F_int):
        super(Attention_block, self).__init__()

        self.W_g = nn.Sequential(
            nn.Conv2d(F_l, F_int, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(F_int)
        )

        self.W_x = nn.Sequential(
            nn.Conv2d(F_g, F_int, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(F_int)
        )

        self.psi = nn.Sequential(
            nn.Conv2d(F_int, 1, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(1),
            nn.Sigmoid()
        )
        
        self.relu = nn.ReLU(inplace=True)

    def forward(self, g, x):
        g1 = self.W_g(g)
        x1 = self.W_x(x)
        psi = self.relu(g1 + x1)
        psi = self.psi(psi)
        out = x * psi
        return out

class AttU_Net(nn.Module):
    """
    Attention Unet implementation
    Paper: https://arxiv.org/abs/1804.03999
    """
    def __init__(self, img_ch=3, output_ch=1):
        super(AttU_Net, self).__init__()
        self.encoder = get_encoder('resnet50', encoder_weights='imagenet')
        n1 = 64
        filters = [n1, n1 * 4, n1 * 8, n1 * 16, n1 * 32]

        self.Maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.Maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.Maxpool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.Maxpool4 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.Conv1 = conv_block(img_ch, filters[0])
        self.Conv2 = conv_block(filters[0], filters[1])
        self.Conv3 = conv_block(filters[1], filters[2])
        self.Conv4 = conv_block(filters[2], filters[3])
        self.Conv5 = conv_block(filters[3], filters[4])

        self.Up5 = up_conv(filters[4], filters[3])
        self.Att5 = Attention_block(F_g=filters[3], F_l=filters[3], F_int=filters[2])
        self.Up_conv5 = conv_block(filters[4], filters[3])

        self.Up4 = up_conv(filters[3], filters[2])
        self.Att4 = Attention_block(F_g=filters[2], F_l=filters[2], F_int=filters[1])
        self.Up_conv4 = conv_block(filters[3], filters[2])

        self.Up3 = up_conv(filters[2], filters[1])
        self.Att3 = Attention_block(F_g=filters[1], F_l=filters[1], F_int=filters[0])
        self.Up_conv3 = conv_block(filters[2], filters[1])

        self.Up2 = up_conv(filters[1], filters[0])
        self.Att2 = Attention_block(F_g=filters[0], F_l=filters[0], F_int=32)
        self.Up_conv2 = conv_block(128, filters[0])
        
        self.Up_last = up_conv(filters[0], filters[0])

        self.Conv = nn.Conv2d(filters[0], output_ch, kernel_size=1, stride=1, padding=0)

        #self.active = torch.nn.Sigmoid()

    def forward(self, x):
        
        e1 = self.Conv1(x)
        #print(e1.shape)
        e2 = self.Maxpool1(e1)
        e2 = self.Conv2(e2)
        #print(e2.shape)
        e3 = self.Maxpool2(e2)
        e3 = self.Conv3(e3)
        #print(e3.shape)
        e4 = self.Maxpool3(e3)
        e4 = self.Conv4(e4)
        #print(e4.shape)
        e5 = self.Maxpool4(e4)
        e5 = self.Conv5(e5)
        #print(e5.shape)
        e5 = self.encoder(x)
        
        #print(e5[0].shape)
        #print(e5[1].shape)
        #print(e5[2].shape)
        #print(e5[3].shape)
        #print(e5[4].shape)
        #print('nita')
        #print(len(e5))
        
        d5 = self.Up5(e5[0])
        x4 = self.Att5(g=d5, x=e5[1])
        d5 = torch.cat((x4, d5), dim=1)
        d5 = self.Up_conv5(d5)
        #print(d5.shape)
        
        d4 = self.Up4(d5)
        x3 = self.Att4(g=d4, x=e5[2])
        d4 = torch.cat((x3, d4), dim=1)       
        d4 = self.Up_conv4(d4)
        #print(d4.shape) #512
        
        d3 = self.Up3(d4)
        #print(d3.shape) #256
        x2 = self.Att3(g=d3, x=e5[3])
        #print(x2.shape) #256
        d3 = torch.cat((x2, d3), dim=1)
        #print(d3.shape) #512
        d3 = self.Up_conv3(d3)
        #print(d3.shape) #256
        
        d2 = self.Up2(d3)
        #print(d2.shape) #64
        x1 = self.Att2(g=d2, x=e5[4])
        #print(e5[4].shape) #64
        #print(x1.shape) #64
        #print(d2.shape) #64
        d2 = torch.cat((x1, d2), dim=1)
        #print(d2.shape) # 128
        d2 = self.Up_conv2(d2)
        
        #out = self.Conv(d2) old line of code
    
        #output is 64,112,112 - 64,224,224
        out = self.Up_last(d2)
        out = self.Conv(out)

        #print(out.shape)

        #out = self.active(out)
        
        return out

class R2AttU_Net(nn.Module):
    """
    Residual Recuurent Block with attention Unet
    Implementation : https://github.com/LeeJunHyun/Image_Segmentation
    """
    def __init__(self, in_ch=3, out_ch=1, t=2):
        super(R2AttU_Net, self).__init__()

        n1 = 64
        filters = [n1, n1 * 2, n1 * 4, n1 * 8, n1 * 16]

        self.Maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.Maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.Maxpool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.Maxpool4 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.RRCNN1 = RRCNN_block(in_ch, filters[0], t=t)
        self.RRCNN2 = RRCNN_block(filters[0], filters[1], t=t)
        self.RRCNN3 = RRCNN_block(filters[1], filters[2], t=t)
        self.RRCNN4 = RRCNN_block(filters[2], filters[3], t=t)
        self.RRCNN5 = RRCNN_block(filters[3], filters[4], t=t)
        
        self.Up5 = up_conv(filters[4], filters[3])
        self.Att5 = Attention_block(F_g=filters[3], F_l=filters[3], F_int=filters[2])
        self.Up_RRCNN5 = RRCNN_block(filters[4], filters[3], t=t)
        
        self.Up4 = up_conv(filters[3], filters[2])
        self.Att4 = Attention_block(F_g=filters[2], F_l=filters[2], F_int=filters[1])
        self.Up_RRCNN4 = RRCNN_block(filters[3], filters[2], t=t)
        
        self.Up3 = up_conv(filters[2], filters[1])
        self.Att3 = Attention_block(F_g=filters[1], F_l=filters[1], F_int=filters[0])
        self.Up_RRCNN3 = RRCNN_block(filters[2], filters[1], t=t)
        
        self.Up2 = up_conv(filters[1], filters[0])
        self.Att2 = Attention_block(F_g=filters[0], F_l=filters[0], F_int=32)
        self.Up_RRCNN2 = RRCNN_block(filters[1], filters[0], t=t)
        
        self.Conv = nn.Conv2d(filters[0], out_ch, kernel_size=1, stride=1, padding=0)

        #self.active = torch.nn.Sigmoid()

    def forward(self, x):

        e1 = self.RRCNN1(x)

        e2 = self.Maxpool1(e1)
        e2 = self.RRCNN2(e2)

        e3 = self.Maxpool2(e2)
        e3 = self.RRCNN3(e3)

        e4 = self.Maxpool3(e3)
        e4 = self.RRCNN4(e4)

        e5 = self.Maxpool4(e4)
        e5 = self.RRCNN5(e5)

        d5 = self.Up5(e5)
        e4 = self.Att5(g=d5, x=e4)
        d5 = torch.cat((e4, d5), dim=1)
        d5 = self.Up_RRCNN5(d5)

        d4 = self.Up4(d5)
        e3 = self.Att4(g=d4, x=e3)
        d4 = torch.cat((e3, d4), dim=1)
        d4 = self.Up_RRCNN4(d4)

        d3 = self.Up3(d4)
        e2 = self.Att3(g=d3, x=e2)
        d3 = torch.cat((e2, d3), dim=1)
        d3 = self.Up_RRCNN3(d3)

        d2 = self.Up2(d3)
        e1 = self.Att2(g=d2, x=e1)
        d2 = torch.cat((e1, d2), dim=1)
        d2 = self.Up_RRCNN2(d2)

        out = self.Conv(d2)

        #out = self.active(out)

        return out

#For nested 3 channels are required

class conv_block_nested(nn.Module):
    
    def __init__(self, in_ch, mid_ch, out_ch):
        super(conv_block_nested, self).__init__()
        self.activation = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_ch, mid_ch, kernel_size=3, padding=1, bias=True)
        self.bn1 = nn.BatchNorm2d(mid_ch)
        self.conv2 = nn.Conv2d(mid_ch, out_ch, kernel_size=3, padding=1, bias=True)
        self.bn2 = nn.BatchNorm2d(out_ch)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.activation(x)
        
        x = self.conv2(x)
        x = self.bn2(x)
        output = self.activation(x)

        return output
    
#Nested Unet

class NestedUNet(nn.Module):
    """
    Implementation of this paper:
    https://arxiv.org/pdf/1807.10165.pdf
    """
    def __init__(self, in_ch=3, out_ch=1):
        super(NestedUNet, self).__init__()
        self.encoder = get_encoder('resnet50', encoder_weights='imagenet')
        n1 = 64
        filters = [n1, n1 * 4, n1 * 8, n1 * 16, n1 * 32]

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.Up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        
        #self.conv0_0 = conv_block_nested(in_ch, filters[0], filters[0])
        #self.conv1_0 = conv_block_nested(filters[0], filters[1], filters[1])
        #self.conv2_0 = conv_block_nested(filters[1], filters[2], filters[2])
        #self.conv3_0 = conv_block_nested(filters[2], filters[3], filters[3])
        #self.conv4_0 = conv_block_nested(filters[3], filters[4], filters[4])
        
        self.conv0_1 = conv_block_nested(filters[0] + filters[1], filters[0], filters[0]) #(64+256, 64, 64)
        self.conv0_2 = conv_block_nested(filters[0]*2 + filters[1], filters[0], filters[0]) #(64*2+256, 64, 64)
        self.conv0_3 = conv_block_nested(filters[0]*3 + filters[1], filters[0], filters[0]) #(64*3+256, 64, 64)
             
        self.conv1_1 = conv_block_nested(filters[1] + filters[2], filters[1], filters[1]) #
        self.conv1_2 = conv_block_nested(filters[1]*2 + filters[2], filters[1], filters[1]) #
        self.conv1_3 = conv_block_nested(filters[1]*3 + filters[2], filters[1], filters[1]) #   
        
        self.conv2_1 = conv_block_nested(filters[2] + filters[3], filters[2], filters[2])
        self.conv2_2 = conv_block_nested(filters[2]*2 + filters[3], filters[2], filters[2])
        
        self.conv3_1 = conv_block_nested(filters[3] + filters[4], filters[3], filters[3])
        
        self.conv0_4 = conv_block_nested(filters[0]*4 + filters[1], filters[0], filters[0])
        
        self.final = nn.Conv2d(filters[0], out_ch, kernel_size=1)
        
    def forward(self, x):
        
        e = self.encoder(x)
        
        #print(e[0].shape) #  1, 2048, 7, 7
        #print(e[1].shape) #  1, 1024, 14, 14
        #print(e[2].shape) #  1, 512, 28, 28
        #print(e[3].shape) #  1, 256, 56, 56
        #print(e[4].shape) #  1, 64, 112, 112 #after upsampling it is 64x224x224
        
        x0_0 = self.Up(e[4])
        x1_0 = e[3]
        x2_0 = e[2]
        x3_0 = e[1]
        x4_0 = e[0]
        
        x0_1 = self.conv0_1(torch.cat([x0_0, self.Up(self.Up(x1_0))], 1))
        x1_1 = self.conv1_1(torch.cat([x1_0, self.Up(x2_0)], 1))
        x2_1 = self.conv2_1(torch.cat([x2_0, self.Up(x3_0)], 1))
        x3_1 = self.conv3_1(torch.cat([x3_0, self.Up(x4_0)], 1))
        
        x2_2 = self.conv2_2(torch.cat([x2_0, x2_1, self.Up(x3_1)], 1))
        
        x1_2 = self.conv1_2(torch.cat([x1_0, x1_1, self.Up(x2_1)], 1))
        x1_3 = self.conv1_3(torch.cat([x1_0, x1_1, x1_2, self.Up(x2_2)], 1))
        
        x0_2 = self.conv0_2(torch.cat([x0_0, x0_1, self.Up(self.Up(x1_1))], 1))
        x0_3 = self.conv0_3(torch.cat([x0_0, x0_1, x0_2, self.Up(self.Up(x1_2))], 1))        
        x0_4 = self.conv0_4(torch.cat([x0_0, x0_1, x0_2, x0_3, self.Up(self.Up(x1_3))], 1))
        
        output = self.final(x0_4)
       
        return output

#Modified Nested Unet with attention module
class Modified_NestedUNet_Att(nn.Module):

    def __init__(self, in_ch=3, out_ch=1):
        super(Modified_NestedUNet_Att, self).__init__()
        
        self.encoder = get_encoder('resnet50', encoder_weights='imagenet')
        
        n1 = 64
        filters = [n1, n1 * 4, n1 * 8, n1 * 16, n1 * 32]
        
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.Up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        
        self.conv0_1 = conv_block_nested(filters[0] + filters[1], filters[0], filters[0]) #(64+256, 64, 64)
        self.conv0_2 = conv_block_nested(filters[0]*2 + filters[1] + filters[2], filters[0], filters[0]) #(64*2+256*2, 64, 64)
        self.conv0_3 = conv_block_nested(filters[0]*3 + filters[1] + filters[2] + filters[3], filters[0], filters[0]) #(64*3+256, 64, 64)
        
        self.conv1_1 = conv_block_nested(filters[1] + filters[2], filters[1], filters[1])
        self.conv1_2 = conv_block_nested(filters[1]*2 + filters[2] + filters[3], filters[1], filters[1])
        self.conv1_3 = conv_block_nested(filters[1]*3 + filters[2]+256, filters[1], filters[1])
        
        self.conv2_1 = conv_block_nested(filters[2] + filters[3], filters[2], filters[2])
        self.conv2_2 = conv_block_nested(filters[2]*2 + filters[3]+512, filters[2], filters[2])
        
        self.conv3_1 = conv_block_nested(filters[3] + filters[4]+1024, filters[3], filters[3])
        
        self.conv0_4 = conv_block_nested(filters[0]*4 + filters[1]+64, filters[0], filters[0])
        
        self.Up5 = up_conv(filters[4], filters[3])
        self.Att5 = Attention_block(F_g=filters[3], F_l=filters[3], F_int=filters[2])
        
        self.Up4 = up_conv(filters[3], filters[2])
        self.Att4 = Attention_block(F_g=filters[2], F_l=filters[2], F_int=filters[1])
        
        self.Up3 = up_conv(filters[2], filters[1])
        self.Att3 = Attention_block(F_g=filters[1], F_l=filters[1], F_int=filters[0])
        
        self.Up2 = up_conv(filters[1], filters[0])
        self.Att2 = Attention_block(F_g=filters[0], F_l=filters[0], F_int=32)
        
        self.final = nn.Conv2d(filters[0], out_ch, kernel_size=1)
        
    def forward(self, x):
        
        e = self.encoder(x)
        
        #print(e[0].shape) #  1, 2048, 7, 7
        #print(e[1].shape) #  1, 1024, 14, 14
        #print(e[2].shape) #  1, 512, 28, 28
        #print(e[3].shape) #  1, 256, 56, 56
        #print(e[4].shape) #  1, 64, 112, 112 #after upsampling it is 64x224x224
        
        x0_0 = self.Up(e[4])
        x1_0 = e[3]
        x2_0 = e[2]
        x3_0 = e[1]
        x4_0 = e[0]
        
        d5 = self.Up5(x4_0)
        x5 = self.Att5(g=d5, x=e[1])

        x0_1 = self.conv0_1(torch.cat([x0_0, self.Up(self.Up(x1_0))], 1))
        x1_1 = self.conv1_1(torch.cat([x1_0, self.Up(x2_0)], 1))
        x2_1 = self.conv2_1(torch.cat([x2_0, self.Up(x3_0)], 1))
        x3_1 = self.conv3_1(torch.cat([x3_0, x5, self.Up(x4_0)], 1))

        d4 = self.Up4(x3_1)
        x4 = self.Att4(g=d4, x=x2_1)
        
        x2_2 = self.conv2_2(torch.cat([x2_0, x2_1, x4, self.Up(x3_1)], 1))
        
        x1_2 = self.conv1_2(torch.cat([x1_0, x1_1, self.Up(x2_1), self.Up(self.Up(x3_0)) ], 1))
        
        d3 = self.Up3(x2_2)
        x3 = self.Att3(g=d3, x=x1_2)

        x1_3 = self.conv1_3(torch.cat([x1_0, x1_1, x1_2, x3, self.Up(x2_2)], 1))
        
        x0_2 = self.conv0_2(torch.cat([x0_0, x0_1, self.Up(self.Up(x1_1)), self.Up(self.Up(self.Up(x2_0)))], 1))
        x0_3 = self.conv0_3(torch.cat([x0_0, x0_1, x0_2, self.Up(self.Up(x1_2)), self.Up(self.Up(self.Up(self.Up(x3_0)))), self.Up(self.Up(self.Up(x2_1)))], 1))

        d2 = self.Up2(x1_3)
        d2 = self.Up(d2)
        x2 = self.Att2(g=d2, x=x0_3)
        
        x0_4 = self.conv0_4(torch.cat([x0_0, x0_1, x0_2, x0_3, x2, self.Up(self.Up(x1_3))], 1))
        
        output = self.final(x0_4)
       
        return output

## -------------------------------------------------------------------------------------------------------------

class conv_block_nested(nn.Module):
    
    def __init__(self, in_ch, mid_ch, out_ch):
        super(conv_block_nested, self).__init__()
        self.activation = nn.ReLU(inplace=True)
        self.dropout = nn.Dropout(0.3)
        self.conv1 = nn.Conv2d(in_ch, mid_ch, kernel_size=3, padding=1, bias=True)
        self.bn1 = nn.BatchNorm2d(mid_ch)
        self.conv2 = nn.Conv2d(mid_ch, out_ch, kernel_size=3, padding=1, bias=True)
        self.bn2 = nn.BatchNorm2d(out_ch)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.activation(x)
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.activation(x)
        output = self.dropout(x)

        return output

# Modified Nested Unet
class  Modified_NestedUNet(nn.Module):
    """
    Implementation of this paper:
    https://arxiv.org/pdf/1807.10165.pdf
    """
    def __init__(self, in_ch=3, out_ch=1):
        super(Modified_NestedUNet, self).__init__()
        
        self.encoder = get_encoder('resnet50', encoder_weights='imagenet')
        
        n1 = 64
        filters = [n1, n1 * 4, n1 * 8, n1 * 16, n1 * 32]
        
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.Up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.Up1 = nn.Upsample(size=224, mode='bilinear', align_corners=True)
        self.Up2 = nn.Upsample(size=56, mode='bilinear', align_corners=True)
        self.Up3 = nn.Upsample(size=28, mode='bilinear', align_corners=True)
        
        self.conv0_1 = conv_block_nested(filters[0] + filters[1], filters[0], filters[0]) #(64+256, 64, 64)
        self.conv0_2 = conv_block_nested(filters[0]*2 + filters[1] + filters[2], filters[0], filters[0]) #(64*2+256*2, 64, 64)
        self.conv0_3 = conv_block_nested(filters[0]*3 + filters[1] + filters[2] + filters[3], filters[0], filters[0]) #(64*3+256, 64, 64)
        
        self.conv1_1 = conv_block_nested(filters[1] + filters[2], filters[1], filters[1])
        self.conv1_2 = conv_block_nested(filters[1]*2 + filters[2] + filters[3], filters[1], filters[1])
        self.conv1_3 = conv_block_nested(filters[1]*3 + filters[2], filters[1], filters[1])
        
        self.conv2_1 = conv_block_nested(filters[2] + filters[3], filters[2], filters[2])
        self.conv2_2 = conv_block_nested(filters[2]*2 + filters[3], filters[2], filters[2])        
        self.conv3_1 = conv_block_nested(filters[3] + filters[4], filters[3], filters[3])
        self.conv0_4 = conv_block_nested(filters[0]*4 + filters[1], filters[0], filters[0])
        
        self.final = nn.Conv2d(filters[0], out_ch, kernel_size=1)
        
    def forward(self, x):
        
        e = self.encoder(x)
        
        x0_0 = self.Up(e[4])
        x1_0 = e[3]
        x2_0 = e[2]
        x3_0 = e[1]
        x4_0 = e[0]
        
        x0_1 = self.conv0_1(torch.cat([x0_0, self.Up1(x1_0)], 1))
        x1_1 = self.conv1_1(torch.cat([x1_0, self.Up(x2_0)], 1))
        x2_1 = self.conv2_1(torch.cat([x2_0, self.Up(x3_0)], 1))
        x3_1 = self.conv3_1(torch.cat([x3_0, self.Up(x4_0)], 1))
        
        x2_2 = self.conv2_2(torch.cat([x2_0, x2_1, self.Up(x3_1)], 1))       
        x1_2 = self.conv1_2(torch.cat([x1_0, x1_1, self.Up2(x2_1), self.Up2(x3_0)], 1))
        x1_3 = self.conv1_3(torch.cat([x1_0, x1_1, x1_2, self.Up(x2_2)], 1))
        
        x0_2 = self.conv0_2(torch.cat([x0_0, x0_1, self.Up1(x1_1), self.Up1(x2_0)], 1))
        x0_3 = self.conv0_3(torch.cat([x0_0, x0_1, x0_2, self.Up1(x1_2), self.Up1(x3_0), self.Up1(x2_1)], 1)) 
        x0_4 = self.conv0_4(torch.cat([x0_0, x0_1, x0_2, x0_3, self.Up1(x1_3)], 1))
        
        output = self.final(x0_4)
       
        return output

class DUpsampling(nn.Module):
    def __init__(self, inplanes, scale, num_class=64, pad=0):
        super(DUpsampling, self).__init__()
        ## W matrix
        self.conv_w = nn.Conv2d(inplanes, num_class * scale * scale, kernel_size=1, padding = pad,bias=False)
        ## P matrix
        self.conv_p = nn.Conv2d(num_class * scale * scale, inplanes, kernel_size=1, padding = pad,bias=False)

        self.scale = scale
    
    def forward(self, x):
        x = self.conv_w(x)
        N, C, H, W = x.size()

        # N, W, H, C
        x_permuted = x.permute(0, 3, 2, 1) 

        # N, W, H*scale, C/scale
        x_permuted = x_permuted.contiguous().view((N, W, H * self.scale, int(C / (self.scale))))

        # N, H*scale, W, C/scale
        x_permuted = x_permuted.permute(0, 2, 1, 3)
        # N, H*scale, W*scale, C/(scale**2)
        x_permuted = x_permuted.contiguous().view((N, W * self.scale, H * self.scale, int(C / (self.scale * self.scale))))

        # N, C/(scale**2), H*scale, W*scale
        x = x_permuted.permute(0, 3, 1, 2)
        
        return x

### Modified NestedUnet_V2
class Modified_NestedUNet_V2(nn.Module):
    """
    Implementation of this paper:
    https://arxiv.org/pdf/1807.10165.pdf
    """
    def __init__(self, in_ch=3, out_ch=1):
        super(Modified_NestedUNet_V2, self).__init__()
        
        self.encoder = get_encoder('resnet50', encoder_weights='imagenet')
        
        n1 = 64
        filters = [n1, n1 * 4, n1 * 8, n1 * 16, n1 * 32]

        # Encoder
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.Up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.Up1 = nn.Upsample(size=224, mode='bilinear', align_corners=True)
        self.Up2 = nn.Upsample(size=56, mode='bilinear', align_corners=True)
        self.Up3 = nn.Upsample(size=28, mode='bilinear', align_corners=True)
        
        self.conv0_1 = conv_block_nested(filters[0] + filters[1], filters[0], filters[0]) #(64+256, 64, 64)
        self.conv0_2 = conv_block_nested(filters[0]*2 + filters[1] + filters[2], filters[0], filters[0]) #(64*2+256*2, 64, 64)
        self.conv0_3 = conv_block_nested(filters[0]*3 + filters[1] + filters[2] + filters[3], filters[0], filters[0]) #(64*3+256, 64, 64)
        
        self.conv1_1 = conv_block_nested(filters[1] + filters[2], filters[1], filters[1])
        self.conv1_2 = conv_block_nested(filters[1]*2 + filters[2] + filters[3], filters[1], filters[1])
        self.conv2_1 = conv_block_nested(filters[2] + filters[3], filters[2], filters[2])
        
        # Decoder
        self.conv1 = nn.Conv2d(filters[4], filters[3], kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(filters[3])
        self.relu = nn.ReLU()
        
        self.conv2 = nn.Conv2d(filters[3], filters[2], kernel_size=3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(filters[2])
        self.dropout2 = nn.Dropout(0.5)
        
        self.conv3 = nn.Conv2d(filters[2], filters[1], kernel_size=3, padding=1, bias=False)
        self.bn3 = nn.BatchNorm2d(filters[1])
        self.dropout3 = nn.Dropout(0.5)
        
        self.conv4 = nn.Conv2d(filters[1], filters[0], kernel_size=3, padding=1, bias=False)
        self.bn4 = nn.BatchNorm2d(filters[0])
        self.dropout4 = nn.Dropout(0.5)
        
        self.dupsample4 = DUpsampling(filters[0], 32, num_class=64)
        
        self.conv6 = nn.Conv2d(filters[0]*2, filters[0], kernel_size=3, padding=1, bias=False)
        self.bn6 = nn.BatchNorm2d(filters[0])
        self.dropout6 = nn.Dropout(0.5)
        
        self.final = nn.Conv2d(filters[0], out_ch, kernel_size=1)
        
    def forward(self, x):
        
        # Encoder
        e = self.encoder(x)
        #print(e[0].shape) #  1, 2048, 7, 7
        #print(e[1].shape) #  1, 1024, 14, 14
        #print(e[2].shape) #  1, 512, 28, 28
        #print(e[3].shape) #  1, 256, 56, 56
        #print(e[4].shape) #  1, 64, 112, 112 #after upsampling it is 64x224x224
        
        x0_0 = self.Up(e[4])
        x1_0 = e[3]
        x2_0 = e[2]
        x3_0 = e[1]
        x4_0 = e[0]
        
        x0_1 = self.conv0_1(torch.cat([x0_0, self.Up1(x1_0)], 1))
        x1_1 = self.conv1_1(torch.cat([x1_0, self.Up(x2_0)], 1))
        x2_1 = self.conv2_1(torch.cat([x2_0, self.Up(x3_0)], 1))
        x1_2 = self.conv1_2(torch.cat([x1_0, x1_1, self.Up2(x2_1), self.Up2(x3_0)], 1))
        x0_2 = self.conv0_2(torch.cat([x0_0, x0_1, self.Up1(x1_1), self.Up1(x2_0)], 1))
        x0_3 = self.conv0_3(torch.cat([x0_0, x0_1, x0_2, self.Up1(x1_2), self.Up1(x3_0), self.Up1(x2_1)], 1)) 
        # x0_3 = 64 x 224 x 224
        
        # Decoder
        x3_1 = self.conv1(x4_0)
        x3_1 = self.bn1(x3_1)
        x3_1 = self.relu(x3_1)
        
        x3_1 = self.conv2(x3_1)
        x3_1 = self.bn2(x3_1)
        x3_1 = self.relu(x3_1)
        x3_1 = self.dropout2(x3_1)
        
        x3_1 = self.conv3(x3_1)
        x3_1 = self.bn3(x3_1)
        x3_1 = self.relu(x3_1)
        x3_1 = self.dropout3(x3_1)
        
        x3_1 = self.conv4(x3_1)
        x3_1 = self.bn4(x3_1)
        x3_1 = self.relu(x3_1)
        x3_1 = self.dropout4(x3_1)
        
        x3_1_up = self.dupsample4(x3_1)

        x0_4_cat = torch.cat((x3_1_up, x0_3), dim=1)
        
        x3_1_final = self.conv6(x0_4_cat)
        x3_1_final = self.bn6(x3_1_final)
        x3_1_final = self.relu(x3_1_final)
        x3_1_final = self.dropout6(x3_1_final)
        
        output = self.final(x3_1_final)
       
        return output
    
class NestedUNet_Old(nn.Module):
    """
    Implementation of this paper:
    https://arxiv.org/pdf/1807.10165.pdf
    """
    def __init__(self, in_ch=3, out_ch=1):
        super(NestedUNet_Old, self).__init__()
        
        self.encoder = get_encoder('resnet50', encoder_weights='imagenet')
        
        n1 = 64
        filters = [n1, n1 * 4, n1 * 8, n1 * 16, n1 * 32]
        
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.Up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        
        self.conv0_1 = conv_block_nested(filters[0] + filters[1], filters[0], filters[0]) #(64+256, 64, 64)
        self.conv0_2 = conv_block_nested(filters[0]*2 + filters[1] + filters[2], filters[0], filters[0]) #(64*2+256*2, 64, 64)
        self.conv0_3 = conv_block_nested(filters[0]*3 + filters[1] + filters[2] + filters[3], filters[0], filters[0]) #(64*3+256, 64, 64)
        
        self.conv1_1 = conv_block_nested(filters[1] + filters[2], filters[1], filters[1])
        self.conv1_2 = conv_block_nested(filters[1]*2 + filters[2] + filters[3], filters[1], filters[1])
        self.conv1_3 = conv_block_nested(filters[1]*3 + filters[2], filters[1], filters[1])
        
        self.conv2_1 = conv_block_nested(filters[2] + filters[3], filters[2], filters[2])
        self.conv2_2 = conv_block_nested(filters[2]*2 + filters[3], filters[2], filters[2])
        
        self.conv3_1 = conv_block_nested(filters[3] + filters[4], filters[3], filters[3])
        
        self.conv0_4 = conv_block_nested(filters[0]*4 + filters[1], filters[0], filters[0])
        
        self.final = nn.Conv2d(filters[0], out_ch, kernel_size=1)
        
    def forward(self, x):
        
        e = self.encoder(x)
        
        #print(e[0].shape) #  1, 2048, 7, 7
        #print(e[1].shape) #  1, 1024, 14, 14
        #print(e[2].shape) #  1, 512, 28, 28
        #print(e[3].shape) #  1, 256, 56, 56
        #print(e[4].shape) #  1, 64, 112, 112 #after upsampling it is 64x224x224
        
        x0_0 = self.Up(e[4])
        x1_0 = e[3]
        x2_0 = e[2]
        x3_0 = e[1]
        x4_0 = e[0]
        
        x0_1 = self.conv0_1(torch.cat([x0_0, self.Up(self.Up(x1_0))], 1))
        x1_1 = self.conv1_1(torch.cat([x1_0, self.Up(x2_0)], 1))
        x2_1 = self.conv2_1(torch.cat([x2_0, self.Up(x3_0)], 1))
        x3_1 = self.conv3_1(torch.cat([x3_0, self.Up(x4_0)], 1))
        
        x2_2 = self.conv2_2(torch.cat([x2_0, x2_1, self.Up(x3_1)], 1))
        
        x1_2 = self.conv1_2(torch.cat([x1_0, x1_1, self.Up(x2_1), self.Up(self.Up(x3_0)) ], 1))
        x1_3 = self.conv1_3(torch.cat([x1_0, x1_1, x1_2, self.Up(x2_2)], 1))
        
        x0_2 = self.conv0_2(torch.cat([x0_0, x0_1, self.Up(self.Up(x1_1)), self.Up(self.Up(self.Up(x2_0)))], 1))
        x0_3 = self.conv0_3(torch.cat([x0_0, x0_1, x0_2, self.Up(self.Up(x1_2)), self.Up(self.Up(self.Up(self.Up(x2_0)))), self.Up(self.Up(self.Up(x2_1)))], 1))        
        x0_4 = self.conv0_4(torch.cat([x0_0, x0_1, x0_2, x0_3, self.Up(self.Up(x1_3))], 1))
        
        output = self.final(x0_4)
       
        return output


In [0]:
#print(AttU_Net(img_ch=3, output_ch=1))

In [0]:
from torchvision.transforms import functional as F
from torch.utils.data.sampler import SubsetRandomSampler

class ImageFolder(data.Dataset):
  
  def __init__(self, root, mode='train'):
    
    self.images_paths = []
    self.GT_paths = []
    self.image_size = 224
    self.mode = mode
    
    if (self.mode == 'train'):
      for folder_path in glob.glob(root):
        for image_path in glob.glob(os.path.join(root + '/ShortVD*', 'ShortVD*[!*.wmv]', 'ShortVD*')):
          self.images_paths.append(image_path)
      for folder_path in glob.glob(root):
        for GT_path in glob.glob(os.path.join(root + '/ShortVD*', 'GT', 'ShortVD*')):
          self.GT_paths.append(GT_path)
    
    if (self.mode == 'test') or (self.mode == 'valid'):
      for folder_path in glob.glob(root):
        for image_path in glob.glob(os.path.join(root + '/testvid*', 'testvid*[!*.wmv]', 'p*')):
          self.images_paths.append(image_path)

      for folder_path in glob.glob(root):
        for GT_path in glob.glob(os.path.join(root + '/testvid*', 'GT', 'p*')):
          self.GT_paths.append(GT_path)

    print("Image count in {} path :{}".format(self.mode, len(self.images_paths)))
  
  def __getitem__(self, index):

    if(self.mode == 'train'):
      # Extract paths corresponding path-image
      image_path = self.images_paths[index]
      splited_path = image_path.split("/")
      image_n = splited_path[-1]
      gt_image_n = image_n[:-len('.tiff')] + '_GT.tiff'
      half_gt_path = '/'.join(splited_path[0:-2])
      GT_path = half_gt_path + '/GT/' + gt_image_n
    
    if(self.mode == 'test') or (self.mode == 'valid'):
      # Extract paths corresponding path-image
      image_path = self.images_paths[index]
      splited_path = image_path.split("/")
      image_n = splited_path[-1]
      gt_image_n = image_n[:-len('.tiff')] + '.tif'
      half_gt_path = '/'.join(splited_path[0:-2])
      GT_path = half_gt_path + '/GT/' + gt_image_n
    
    image = Image.open(image_path)
    GT = Image.open(GT_path)
    
    p_transform = random.random()
    
    if (self.mode == 'train'):
        
        ###------- Affine transformations performed & saved before training
        """
        ### Shear & Translate
        image = F.affine(image, 0, translate=[-1, 1], scale=1, shear=25, resample=False, fillcolor=0)
        GT = F.affine(GT, 0, translate=[1, 1], scale=1, shear=25, resample=False, fillcolor=0)
        
        ### Rotate
        image = F.rotate(image, 350, resample=False, expand=False, center=None)
        GT = F.rotate(GT, 350, resample=False, expand=False, center=None)
    
        image = F.rotate(image, 10, resample=False, expand=True, center=None)
        GT = F.rotate(GT, 10, resample=False, expand=True, center=None)
        """
        
        ###----- Perspective transformation
        if random.random() < 0.5:
          image = F.hflip(image)
          GT = F.hflip(GT)
        
        if random.random() < 0.4:
          image = F.vflip(image)
          GT = F.vflip(GT)
    
    image = image.resize((224,224), Image.ANTIALIAS)
    image = np.array(image, dtype='uint8')
    image = torch.from_numpy(image)
    image = image.permute(2,0,1)
    image = image.float()/255
    
    GT = GT.resize((224,224), Image.ANTIALIAS)
    GT = np.array(GT, dtype='uint8')
    GT = torch.from_numpy(GT)
    GT [GT > 0] = 255.0
    GT = GT.float()/255.0
    
    #unique, counts = np.unique(GT, return_counts=True)
    #print(dict(zip(unique, counts)))
    
    Norm_ = T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    image = Norm_(image)
    
    return image, GT

  def __len__(self):
    #Returns the total number of font files
    return len(self.images_paths)

train_dataset = ImageFolder(root='/media/user3/My Passport/Nita/TrainingSet_NewGT', mode='train')
valid_dataset = ImageFolder(root='/media/user3/My Passport/Nita/ValidationSet_NewGT', mode='valid')
test_dataset = ImageFolder(root='/media/user3/My Passport/Nita/AsuMayoTest', mode='test')

train_loader = data.DataLoader(dataset=train_dataset, batch_size=6, shuffle=True, num_workers=8)
valid_loader = data.DataLoader(dataset=valid_dataset, batch_size=6, shuffle=True, num_workers=8)
test_loader = data.DataLoader(dataset=test_dataset, batch_size=6, shuffle=True, num_workers=8)

"""
dataset_size = len(train_dataset)
validation_split = 0.2
random_seed= 30
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
np.random.seed(random_seed)
np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = data.DataLoader(dataset=train_dataset, batch_size=14, sampler=train_sampler, num_workers=8)
valid_loader = data.DataLoader(dataset=train_dataset, batch_size=14, sampler=valid_sampler, num_workers=8)
test_loader = data.DataLoader(dataset=test_dataset, batch_size=14, shuffle=False, num_workers=8)
"""

Image count in train path :22757
Image count in valid path :4820
Image count in test path :17574


'\ndataset_size = len(train_dataset)\nvalidation_split = 0.2\nrandom_seed= 30\nindices = list(range(dataset_size))\nsplit = int(np.floor(validation_split * dataset_size))\nnp.random.seed(random_seed)\nnp.random.shuffle(indices)\ntrain_indices, val_indices = indices[split:], indices[:split]\n\ntrain_sampler = SubsetRandomSampler(train_indices)\nvalid_sampler = SubsetRandomSampler(val_indices)\n\ntrain_loader = data.DataLoader(dataset=train_dataset, batch_size=14, sampler=train_sampler, num_workers=8)\nvalid_loader = data.DataLoader(dataset=train_dataset, batch_size=14, sampler=valid_sampler, num_workers=8)\ntest_loader = data.DataLoader(dataset=test_dataset, batch_size=14, shuffle=False, num_workers=8)\n'

In [0]:
def plot_images(images, cls_true, cls_pred=None, smooth=True, filename='test.png'):
    #pdb.set_trace()
    assert len(images) == len(cls_true)
    fig, axes = plt.subplots(5, 5,figsize=(60, 60))
   
    if cls_pred is None:
        hspace = 0.6
    else:
        hspace = 0.9

    fig.subplots_adjust(hspace=hspace, wspace=0.3)

    # Interpolation type.
    if smooth:
        interpolation = 'spline16'
    else:
        interpolation = 'nearest'

    count1 =0
    count2 =0
    for i, ax in enumerate(axes.flat):
        if i < len(images)*2:
            # Plot image.
            if i % 2 ==0:
                ax.imshow(np.uint8(images[count1]),interpolation=interpolation)
                count1+= 1
            else:
                ax.imshow(np.uint8(cls_true[count2]),interpolation=interpolation,cmap=plt.get_cmap('gray'))
                count2+= 1
        ax.set_xticks([])
        ax.set_yticks([])
  #  plt.rcParams["figure.figsize"] = (60,60)
    plt.savefig(filename,dpi=100)
    plt.show()

In [0]:
image, gt = next(iter(train_loader));

In [0]:

### Plot only one image
def im_convert(tensor):
    image = tensor.cpu().clone().detach().numpy()
    image = image.transpose(1,2,0)
    image = image*np.array((0.5, 0.5, 0.5)) + np.array((0.5, 0.5, 0.5))
    image = image.clip(0, 1)
    return image

#plt.imshow(im_convert(image[0]))


In [0]:
image = image.permute(0,2,3,1).numpy()
image = image*np.array((0.5, 0.5, 0.5)) + np.array((0.5, 0.5, 0.5))

### Shfaq disa foto nga dataset
#plot_images(image*255, gt.numpy()*255, cls_pred=None, smooth=True)

In [0]:
### Plot gradients

from matplotlib.lines import Line2D

def plot_grad_flow(named_parameters):
    '''Plots the gradients flowing through different layers in the net during training.
    Can be used for checking for possible gradient vanishing / exploding problems.
    
    Usage: Plug this function in Trainer class after loss.backwards() as 
    "plot_grad_flow(self.model.named_parameters())" to visualize the gradient flow'''
    ave_grads = []
    max_grads= []
    layers = []
    for n, p in named_parameters:
        if(p.requires_grad) and ("bias" not in n):
            layers.append(n)
            ave_grads.append(p.grad.abs().mean())
            max_grads.append(p.grad.abs().max())
    plt.bar(np.arange(len(max_grads)), max_grads, alpha=0.1, lw=1, color="c")
    plt.bar(np.arange(len(max_grads)), ave_grads, alpha=0.1, lw=1, color="b")
    plt.hlines(0, 0, len(ave_grads)+1, lw=2, color="k" )
    plt.xticks(range(0,len(ave_grads), 1), layers, rotation="vertical")
    plt.xlim(left=0, right=len(ave_grads))
    plt.ylim(bottom = -0.001, top=0.02) # zoom in on the lower gradient regions
    plt.xlabel("Layers")
    plt.ylabel("average gradient")
    plt.title("Gradient flow")
    plt.grid(True)
    plt.legend([Line2D([0], [0], color="c", lw=4),
                Line2D([0], [0], color="b", lw=4),
                Line2D([0], [0], color="k", lw=4)], ['max-gradient', 'mean-gradient', 'zero-gradient'])

In [0]:
#======================= Detection evaluation =========================#

# 1 indicates polyp; 0 indicates non-polyp
def perimage(SR, GT):

    tpd = fnd = fpd = tnd = 0

    polyp_in_SR = 0 if sum(SR.flatten()) == 0 else 1
    polyp_in_GT = 0 if sum(GT.flatten()) == 0 else 1
    #print(sum(GT.flatten()))
    #print(sum(SR.flatten()))
    for i in range(1, polyp_in_SR + 1):
        intersection = np.sum((SR[GT == 1] == i))
        if intersection > 0:
            tpd += 1
        else:
            fpd += 1

    for i in range(1, polyp_in_GT + 1):
        intersection = np.sum((GT[SR == 1] == i))
        if intersection == 0:
            fnd += 1

    if polyp_in_GT == polyp_in_SR == 0:
        tnd += 1

    return tpd, tnd, fpd, fnd

def detection_rate(SR, GT):
    
    SR.flatten()
    GT.flatten()
    SR = np.where(SR > 0.5, 1, 0)
    GT = np.where(GT > 0, 1, 0)

    SR = np.array(SR, dtype=np.double)
    GT = np.array(GT, dtype=np.double)

    #pdb.set_trace() 
    tpd = fnd = fpd = tnd = 0

    for sr, gt in zip(SR, GT):
        #pdb.set_trace()
        tpdi, tndi, fpdi, fndi = perimage(sr, gt)
        tpd = tpd + tpdi
        fnd = fnd + fndi
        tnd = tnd + tndi
        fpd = fpd + fpdi

    return tpd, tnd, fpd, fnd

def detection_accuracy(tpd, tnd, fpd, fnd):
    return ((tnd + tpd) / (tnd + tpd + fnd + fpd + 1e-20))

def detection_recall(tpd, fnd):
    return ((tpd) / (tpd + fnd + 1e-20))

def detection_precision(tpd, fpd):
    return ((tpd) / (tpd + fpd + 1e-20))

def detection_dice_coefficient(tpd, fnd, fpd):
    return ((2*tpd)/(2*tpd + fnd + fpd + 1e-20))

#======================= Segmentation evaluation =======================#

def segmentation_rate(SR, GT):
    
    SR.flatten()
    GT.flatten()

    SR = np.where(SR > 0.5, 1, 0)
    GT = np.where(GT > 0, 1, 0)

    SR = np.array(SR, dtype=np.double)
    GT = np.array(GT, dtype=np.double)

    #pdb.set_trace()
    tp = np.sum(SR[GT == 1] == 1)
    fp = np.sum(SR[GT == 0] == 1)
    fn = np.sum(SR[GT == 1] == 0)
    tn = np.sum(SR[GT == 0] == 0)
    return tp, tn, fp, fn

def segmentation_accuracy(tp, tn, fp, fn):
    return ((tp + tn)/(tp + tn + fp + fn + 1e-20))

def segmentation_recall(tp, fn):
    return ((tp)/(tp + fn + 1e-20))

def segmentation_precision(tp, fp):
    return ((tp) / (tp + fp + 1e-20))

def segmentation_dice_coefficient(tp, fn, fp):
    return ((2*tp)/(2*tp + fn + fp + 1e-20))


In [0]:
### EORROR: Foscal Loss + Dice loss
"""
def dice_loss(input, target):
    #input = torch.sigmoid(input)
    smooth = 1.0
    input = input.squeeze()
    target = target.squeeze()
    iflat = input.view(-1)
    tflat = target.view(-1)
    intersection = (iflat * tflat).sum()
    
    return ((2.0 * intersection + smooth) / (iflat.sum() + tflat.sum() + smooth))
  
class FocalLoss(nn.Module):
    def __init__(self, gamma):
        super().__init__()
        self.gamma = gamma
        
    def forward(self, input, target):
        if not (target.size() == input.size()):
            raise ValueError("Target size ({}) must be the same as input size ({})"
                             .format(target.size(), input.size()))
        input = input.squeeze()
        target = target.squeeze()
        max_val = (-input).clamp(min=0)
        loss = input - input * target + max_val + \
            ((-max_val).exp() + (-input - max_val).exp()).log()

        invprobs = torch.nn.functional.logsigmoid(-input * (target * 2.0 - 1.0))
        loss = (invprobs * self.gamma).exp() * loss
        
        return loss.mean()

class MixedLoss(nn.Module):
    def __init__(self, alpha, gamma):
        super().__init__()
        self.alpha = alpha
        self.focal = FocalLoss(gamma)Polyp
        
    def forward(self, input, target):
        input = input.squeeze()
        target = target.squeeze()
        loss = self.alpha*self.focal(input, target) - torch.log(dice_loss(input, target))
        return loss.mean()
   
### EORROR: BCE + DICE LOSS

### Error loss function
def bce_dice_loss(SR, GT):
  
    SR = SR.squeeze()
    GT = GT.squeeze()
    iflat = SR.view(-1)
    tflat = GT.view(-1)
    
    #pdb.set_trace()
    
    criterion = torch.nn.BCELoss()
    loss = criterion(iflat, tflat) + dice_loss(SR, GT)
    return loss
"""

def dice_loss(SR, GT):
    #input = torch.sigmoid(input)
    smooth = 1.
  
    SR = SR.squeeze()
    GT = GT.squeeze()
    SR_probs = torch.nn.functional.sigmoid(SR)
    iflat = SR_probs.view(-1)
    tflat = GT.view(-1)
    
    intersection = (iflat * tflat).sum()
    
    return 1 - ((2. * intersection + smooth) / (iflat.sum() + tflat.sum() + smooth))

def weighted_bce(SR, GT):
    
    SR = SR.squeeze()
    GT = GT.squeeze()
    
    iflat = SR.view(-1)
    tflat = GT.view(-1)
    SR_dim = iflat.shape
    
    pos_weight = 100*torch.ones((SR_dim)).view(-1).cuda()
    pos_weight = pos_weight.squeeze()
    
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
    
    loss = criterion(iflat, tflat) + dice_loss(SR, GT)
    
    return loss

def R_custom_penalization(SR, GT, limit=1.2):
        
    SR = (np.squeeze(SR)).cpu().detach().numpy()
    GT = (np.squeeze(GT)).cpu().detach().numpy()
    
    SR = np.array(SR, dtype=np.double)
    GT = np.array(GT, dtype=np.double)
    
    SR = np.where(SR > 0.5, 1, 0)
    GT = np.where(GT > 0, 1, 0)

    value = 1
    # cuda = torch.device('cuda')
    # *torch.tensor(R_custom_penalization(iflat, tflat, limit=1.2)).cuda()
    polyp_in_SR = 0 if sum(SR.flatten()) == 0 else 1
    polyp_in_GT = 0 if sum(GT.flatten()) == 0 else 1
    
    for i in range(1, polyp_in_GT + 1):
        intersection = np.sum((GT[SR == 1] == i))
        if intersection == 0:
            value = limit 

    return value 

In [0]:
#!/home/nita/anaconda3/lin/python3.7

class Solver(object):
  
  def __init__(self, train_loader, valid_loader, test_loader):
    
    # Data loader
    self.train_loader = train_loader
    self.valid_loader = valid_loader
    self.test_loader = test_loader
    
    # Models
    self.attUnet = None
    self.img_ch = 3
    self.output_ch = 1
    self.checkpoint_epoch = 0
    
    # Hyper-parameters
    self.lr = 0.0001
    self.beta1 = 0.9
    self.beta2 =0.999
    
    # Training settings
    self.num_epochs = 3
    self.batch_size = 6
    
    # Paths
    self.model_path = '/home/user3/myenv/models'
    self.checkpoint_path = '/home/user3/myenv/checkpoints/checkpoint.pt'
    
    self.mode = 'train'
    self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    self.model_type = 'Modified_NestedUNet_V2'
    self.t = 2
    self.build_model()

    #weight_initialization = init_weights(self.attUnet, init_type='xavier', gain=0.02)
    
    #self.early_stopping = EarlyStopping(patience=10, verbose=True)
    
    #attUnet_path = os.path.join(self.model_path, 'Model_ R2AttU_Net-50-0.0001-4.pkl')
    #self.attUnet.load_state_dict(torch.load(attUnet_path))
    #print('%s is Successfully Loaded from %s' % (self.model_type, attUnet_path))
    
    #self.attUnet = self.load_checkpoint(self.checkpoint_path)
    
  def build_model(self):

    if self.model_type == 'U_Net':
      self.attUnet = nn.DataParallel(U_Net(in_ch=3, out_ch=1))
    if self.model_type == 'Net':
        self.attUnet = Net()
    elif self.model_type == 'AttU_Net':
      self.attUnet = AttU_Net(img_ch=3, output_ch=1)
    elif self.model_type == 'R2AttU_Net':
      self.attUnet = R2AttU_Net(in_ch=3, out_ch=1, t=self.t)
    elif self.model_type == 'NestedUNet':
      self.attUnet =  nn.DataParallel(NestedUNet(in_ch=3, out_ch=1))
    elif self.model_type == 'Modified_NestedUNet':
      self.attUnet =  nn.DataParallel(Modified_NestedUNet(in_ch=3, out_ch=1))
    elif self.model_type == 'Modified_NestedUNet_Att':
      self.attUnet =  nn.DataParallel(Modified_NestedUNet_Att(in_ch=3, out_ch=1))
    elif self.model_type == 'Modified_NestedUNet_V2':
      self.attUnet =  nn.DataParallel(Modified_NestedUNet_V2(in_ch=3, out_ch=1))
    elif self.model_type == 'NestedUNet_Old':
      self.attUnet =  nn.DataParallel(NestedUNet_Old(in_ch=3, out_ch=1))
    elif self.model_type == 'U_Net_noskip':
      self.attUnet =  nn.DataParallel(U_Net_noskip(in_ch=3, out_ch=1))
    
    """
    my_list = ['self.attUnet.module.encoder.conv1.weight'
    'self.attUnet.module.encoder.bn1.weight'
    'self.attUnet.module.encoder.bn1.bias'
    'self.attUnet.module.encoder.layer1.0.conv1.weight'
    'self.attUnet.module.encoder.layer1.0.bn1.weight'
    'self.attUnet.module.encoder.layer1.0.bn1.bias'
    'self.attUnet.module.encoder.layer1.0.conv2.weight'
    'self.attUnet.module.encoder.layer1.0.bn2.weight'
    'self.attUnet.module.encoder.layer1.0.bn2.bias'
    'self.attUnet.module.encoder.layer1.0.conv3.weight'
    'self.attUnet.module.encoder.layer1.0.bn3.weight'
    'self.attUnet.module.encoder.layer1.0.bn3.bias'
    'self.attUnet.module.encoder.layer1.0.downsample.0.weight'
    'self.attUnet.module.encoder.layer1.0.downsample.1.weight'
    'self.attUnet.module.encoder.layer1.0.downsample.1.bias'
    'self.attUnet.module.encoder.layer1.1.conv1.weight'
    'self.attUnet.module.encoder.layer1.1.bn1.weight'
    'self.attUnet.module.encoder.layer1.1.bn1.bias'
    'self.attUnet.module.encoder.layer1.1.conv2.weight'
    'self.attUnet.module.encoder.layer1.1.bn2.weight'
    'self.attUnet.module.encoder.layer1.1.bn2.bias'
    'self.attUnet.module.encoder.layer1.1.conv3.weight'
    'self.attUnet.module.encoder.layer1.1.bn3.weight'
    'self.attUnet.module.encoder.layer1.1.bn3.bias'
    'self.attUnet.module.encoder.layer1.2.conv1.weight'
    'self.attUnet.module.encoder.layer1.2.bn1.weight'
    'self.attUnet.module.encoder.layer1.2.bn1.bias'
    'self.attUnet.module.encoder.layer1.2.conv2.weight'
    'self.attUnet.module.encoder.layer1.2.bn2.weight'
    'self.attUnet.module.encoder.layer1.2.bn2.bias'
    'self.attUnet.module.encoder.layer1.2.conv3.weight'
    'self.attUnet.module.encoder.layer1.2.bn3.weight'
    'self.attUnet.module.encoder.layer1.2.bn3.bias'
    'self.attUnet.module.encoder.layer2.0.conv1.weight'
    'self.attUnet.module.encoder.layer2.0.bn1.weight'
    'self.attUnet.module.encoder.layer2.0.bn1.bias'
    'self.attUnet.module.encoder.layer2.0.conv2.weight'
    'self.attUnet.module.encoder.layer2.0.bn2.weight'
    'self.attUnet.module.encoder.layer2.0.bn2.bias'
    'self.attUnet.module.encoder.layer2.0.conv3.weight'
    'self.attUnet.module.encoder.layer2.0.bn3.weight'
    'self.attUnet.module.encoder.layer2.0.bn3.bias'
    'self.attUnet.module.encoder.layer2.0.downsample.0.weight'
    'self.attUnet.module.encoder.layer2.0.downsample.1.weight'
    'self.attUnet.module.encoder.layer2.0.downsample.1.bias'
    'self.attUnet.module.encoder.layer2.1.conv1.weight'
    'self.attUnet.module.encoder.layer2.1.bn1.weight'
    'self.attUnet.module.encoder.layer2.1.bn1.bias'
    'self.attUnet.module.encoder.layer2.1.conv2.weight'
    'self.attUnet.module.encoder.layer2.1.bn2.weight'
    'self.attUnet.module.encoder.layer2.1.bn2.bias'
    'self.attUnet.module.encoder.layer2.1.conv3.weight'
    'self.attUnet.module.encoder.layer2.1.bn3.weight'
    'self.attUnet.module.encoder.layer2.1.bn3.bias'
    'self.attUnet.module.encoder.layer2.2.conv1.weight'
    'self.attUnet.module.encoder.layer2.2.bn1.weight'
    'self.attUnet.module.encoder.layer2.2.bn1.bias'
    'self.attUnet.module.encoder.layer2.2.conv2.weight'
    'self.attUnet.module.encoder.layer2.2.bn2.weight'
    'self.attUnet.module.encoder.layer2.2.bn2.bias'
    'self.attUnet.module.encoder.layer2.2.conv3.weight'
    'self.attUnet.module.encoder.layer2.2.bn3.weight'
    'self.attUnet.module.encoder.layer2.2.bn3.bias'
    'self.attUnet.module.encoder.layer2.3.conv1.weight'
    'self.attUnet.module.encoder.layer2.3.bn1.weight'
    'self.attUnet.module.encoder.layer2.3.bn1.bias'
    'self.attUnet.module.encoder.layer2.3.conv2.weight'
    'self.attUnet.module.encoder.layer2.3.bn2.weight'
    'self.attUnet.module.encoder.layer2.3.bn2.bias'
    'self.attUnet.module.encoder.layer2.3.conv3.weight'
    'self.attUnet.module.encoder.layer2.3.bn3.weight'
    'self.attUnet.module.encoder.layer2.3.bn3.bias'
    'self.attUnet.module.encoder.layer3.0.conv1.weight'
    'self.attUnet.module.encoder.layer3.0.bn1.weight'
    'self.attUnet.module.encoder.layer3.0.bn1.bias'
    'self.attUnet.module.encoder.layer3.0.conv2.weight'
    'self.attUnet.module.encoder.layer3.0.bn2.weight'
    'self.attUnet.module.encoder.layer3.0.bn2.bias'
    'self.attUnet.module.encoder.layer3.0.conv3.weight'
    'self.attUnet.module.encoder.layer3.0.bn3.weight'
    'self.attUnet.module.encoder.layer3.0.bn3.bias'
    'self.attUnet.module.encoder.layer3.0.downsample.0.weight'
    'self.attUnet.module.encoder.layer3.0.downsample.1.weight'
    'self.attUnet.module.encoder.layer3.0.downsample.1.bias'
    'self.attUnet.module.encoder.layer3.1.conv1.weight'
    'self.attUnet.module.encoder.layer3.1.bn1.weight'
    'self.attUnet.module.encoder.layer3.1.bn1.bias'
    'self.attUnet.module.encoder.layer3.1.conv2.weight'
    'self.attUnet.module.encoder.layer3.1.bn2.weight'
    'self.attUnet.module.encoder.layer3.1.bn2.bias'
    'self.attUnet.module.encoder.layer3.1.conv3.weight'
    'self.attUnet.module.encoder.layer3.1.bn3.weight'
    'self.attUnet.module.encoder.layer3.1.bn3.bias'
    'self.attUnet.module.encoder.layer3.2.conv1.weight'
    'self.attUnet.module.encoder.layer3.2.bn1.weight'
    'self.attUnet.module.encoder.layer3.2.bn1.bias'
    'self.attUnet.module.encoder.layer3.2.conv2.weight'
    'self.attUnet.module.encoder.layer3.2.bn2.weight'
    'self.attUnet.module.encoder.layer3.2.bn2.bias'
    'self.attUnet.module.encoder.layer3.2.conv3.weight'
    'self.attUnet.module.encoder.layer3.2.bn3.weight'
    'self.attUnet.module.encoder.layer3.2.bn3.bias'
    'self.attUnet.module.encoder.layer3.3.conv1.weight'
    'self.attUnet.module.encoder.layer3.3.bn1.weight'
    'self.attUnet.module.encoder.layer3.3.bn1.bias'
    'self.attUnet.module.encoder.layer3.3.conv2.weight'
    'self.attUnet.module.encoder.layer3.3.bn2.weight'
    'self.attUnet.module.encoder.layer3.3.bn2.bias'
    'self.attUnet.module.encoder.layer3.3.conv3.weight'
    'self.attUnet.module.encoder.layer3.3.bn3.weight'
    'self.attUnet.module.encoder.layer3.3.bn3.bias'
    'self.attUnet.module.encoder.layer3.4.conv1.weight'
    'self.attUnet.module.encoder.layer3.4.bn1.weight'
    'self.attUnet.module.encoder.layer3.4.bn1.bias'
    'self.attUnet.module.encoder.layer3.4.conv2.weight'
    'self.attUnet.module.encoder.layer3.4.bn2.weight'
    'self.attUnet.module.encoder.layer3.4.bn2.bias'
    'self.attUnet.module.encoder.layer3.4.conv3.weight'
    'self.attUnet.module.encoder.layer3.4.bn3.weight'
    'self.attUnet.module.encoder.layer3.4.bn3.bias'
    'self.attUnet.module.encoder.layer3.5.conv1.weight'
    'self.attUnet.module.encoder.layer3.5.bn1.weight'
    'self.attUnet.module.encoder.layer3.5.bn1.bias'
    'self.attUnet.module.encoder.layer3.5.conv2.weight'
    'self.attUnet.module.encoder.layer3.5.bn2.weight'
    'self.attUnet.module.encoder.layer3.5.bn2.bias'
    'self.attUnet.module.encoder.layer3.5.conv3.weight'
    'self.attUnet.module.encoder.layer3.5.bn3.weight'
    'self.attUnet.module.encoder.layer3.5.bn3.bias'
    'self.attUnet.module.encoder.layer4.0.conv1.weight'
    'self.attUnet.module.encoder.layer4.0.bn1.weight'
    'self.attUnet.module.encoder.layer4.0.bn1.bias'
    'self.attUnet.module.encoder.layer4.0.conv2.weight'
    'self.attUnet.module.encoder.layer4.0.bn2.weight'
    'self.attUnet.module.encoder.layer4.0.bn2.bias'
    'self.attUnet.module.encoder.layer4.0.conv3.weight'
    'self.attUnet.module.encoder.layer4.0.bn3.weight'
    'self.attUnet.module.encoder.layer4.0.bn3.bias'
    'self.attUnet.module.encoder.layer4.0.downsample.0.weight'
    'self.attUnet.module.encoder.layer4.0.downsample.1.weight'
    'self.attUnet.module.encoder.layer4.0.downsample.1.bias'
    'self.attUnet.module.encoder.layer4.1.conv1.weight'
    'self.attUnet.module.encoder.layer4.1.bn1.weight'
    'self.attUnet.module.encoder.layer4.1.bn1.bias'
    'self.attUnet.module.encoder.layer4.1.conv2.weight'
    'self.attUnet.module.encoder.layer4.1.bn2.weight'
    'self.attUnet.module.encoder.layer4.1.bn2.bias'
    'self.attUnet.module.encoder.layer4.1.conv3.weight'
    'self.attUnet.module.encoder.layer4.1.bn3.weight'
    'self.attUnet.module.encoder.layer4.1.bn3.bias'
    'self.attUnet.module.encoder.layer4.2.conv1.weight'
    'self.attUnet.module.encoder.layer4.2.bn1.weight'
    'self.attUnet.module.encoder.layer4.2.bn1.bias'
    'self.attUnet.module.encoder.layer4.2.conv2.weight'
    'self.attUnet.module.encoder.layer4.2.bn2.weight'
    'self.attUnet.module.encoder.layer4.2.bn2.bias'
    'self.attUnet.module.encoder.layer4.2.conv3.weight'
    'self.attUnet.module.encoder.layer4.2.bn3.weight'
    'self.attUnet.module.encoder.layer4.2.bn3.bias']
    
    encoder_params = list(filter(lambda kv: kv[0] in my_list, self.attUnet.named_parameters()))
      
    self.optimizer = optim.Adam([
    {"params": encoder_params, "lr": 0.00001},
    { "params" : self.attUnet.module.conv0_1.conv1.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv0_1.conv1.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv0_1.bn1.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv0_1.bn1.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv0_1.conv2.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv0_1.conv2.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv0_1.bn2.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv0_1.bn2.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv0_2.conv1.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv0_2.conv1.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv0_2.bn1.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv0_2.bn1.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv0_2.conv2.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv0_2.conv2.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv0_2.bn2.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv0_2.bn2.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv0_3.conv1.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv0_3.conv1.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv0_3.bn1.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv0_3.bn1.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv0_3.conv2.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv0_3.conv2.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv0_3.bn2.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv0_3.bn2.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv1_1.conv1.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv1_1.conv1.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv1_1.bn1.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv1_1.bn1.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv1_1.conv2.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv1_1.conv2.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv1_1.bn2.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv1_1.bn2.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv1_2.conv1.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv1_2.conv1.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv1_2.bn1.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv1_2.bn1.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv1_2.conv2.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv1_2.conv2.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv1_2.bn2.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv1_2.bn2.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv1_3.conv1.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv1_3.conv1.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv1_3.bn1.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv1_3.bn1.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv1_3.conv2.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv1_3.conv2.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv1_3.bn2.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv1_3.bn2.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv2_1.conv1.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv2_1.conv1.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv2_1.bn1.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv2_1.bn1.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv2_1.conv2.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv2_1.conv2.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv2_1.bn2.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv2_1.bn2.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv2_2.conv1.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv2_2.conv1.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv2_2.bn1.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv2_2.bn1.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv2_2.conv2.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv2_2.conv2.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv2_2.bn2.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv2_2.bn2.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv3_1.conv1.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv3_1.conv1.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv3_1.bn1.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv3_1.bn1.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv3_1.conv2.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv3_1.conv2.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv3_1.bn2.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv3_1.bn2.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv0_4.conv1.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv0_4.conv1.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv0_4.bn1.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv0_4.bn1.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv0_4.conv2.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv0_4.conv2.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv0_4.bn2.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv0_4.bn2.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.final.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.final.bias, "lr": 0.0001}], self.lr, [self.beta1, self.beta2])
    """
    # When you want to update only a part of weights
    #self.optimizer = optim.Adam(filter(lambda p: p.requires_grad, self.attUnet.parameters()), self.lr, [self.beta1, self.beta2])
    
    #When you want to update default -NORMAL OPTIMIZATION
    self.optimizer = optim.Adam(list(self.attUnet.parameters()), self.lr, [self.beta1, self.beta2])
    self.attUnet.to(self.device)
    self.print_network(self.attUnet, self.model_type)
   
  #Print out the network information
  def print_network(self, model, name):
    num_params = 0
    for p in model.parameters():
      num_params += p.numel()
        #print(model)
    for name, param in model.named_parameters():
        if param.requires_grad:
            print(name)
    print(name)
    print("The number of parameters: {}".format(num_params))

  def load_checkpoint(self, file_path):
    
    checkpoint = torch.load(file_path)
    self.checkpoint_epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    self.attUnet.load_state_dict(checkpoint['model_state_dict'])
    self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    
    print("Checkpoint was loaded")
    print(loss)
    print(self.checkpoint_epoch)
    
    """
    for parameter in model.parameters():
      parameter.requires_grad = False

    model.eval()
    """
    return self.attUnet

  def train(self):
    ### Train encoder, generator and discriminator
    # range(self.checkpoint_epoch, self.num_epochs)
    #range(self.num_epochs) - original

    for epoch in range(self.checkpoint_epoch, self.num_epochs):
      self.per_epoch_train(epoch)
      #try:
      self.per_epoch_valid(epoch)
      #except EarlyStopError:
       # break

  def per_epoch_train(self,epoch):

    avg_meter = defaultdict(float)
    pbar = tqdm(enumerate(self.train_loader), total=len(self.train_loader), desc="Epoch {}".format(epoch), ncols=0)
    self.attUnet.train(True)
    meter = {}
    
    acc_detection = 0.
    recall_detection = 0.
    precision_detection = 0.
    DC_detection = 0.
    
    acc_segmentation = 0.
    recall_segmentation = 0.
    precision_segmentation = 0.
    DC_segmentation = 0.
    
    length = 0
    avg_loss_train = 0.
    iterations = 0

    for i, data in pbar:
    
      images = data[0]
      GT = data[1]
      
      images, GT = Variable(images.cuda()), Variable(GT.cuda())
      SR = self.attUnet(images)

      length += images.size(0)      
      iterations += 1
      
      #criterion = MixedLoss(alpha=0.25, gamma=2)
      #loss = criterion(SR_probs, GT)
      
      #loss = bce_dice_loss(SR_probs, GT)

      loss = weighted_bce(SR, GT)
      avg_loss_train += loss.item()
      
      self.optimizer.zero_grad()
      loss.backward()
      self.optimizer.step()
      plot_grad_flow(self.attUnet.named_parameters())
        
      SR_probs = torch.nn.functional.sigmoid(SR)
      SR_numpy = (np.squeeze(SR_probs).data).cpu().numpy()
      GT_numpy = (np.squeeze(GT).data).cpu().numpy()
      
      #print(GT_numpy.shape)
      #print(SR_numpy.shape)
      
      #print(GT_numpy.size)
      #print(SR_numpy.size)
      
      #pdb.set_trace()
      tpd, tnd, fpd, fnd = detection_rate(SR_numpy, GT_numpy)
      tp, tn, fp, fn = segmentation_rate(SR_numpy, GT_numpy)
      
      acc_detection += detection_accuracy(tpd, tnd, fpd, fnd)
      recall_detection += detection_recall(tpd, fnd)
      precision_detection += detection_precision(tpd, fpd)
      DC_detection += detection_dice_coefficient(tpd, fnd, fpd)
      
      acc_segmentation += segmentation_accuracy(tp, tn, fp, fn)
      recall_segmentation += segmentation_recall(tp, fn)
      precision_segmentation += segmentation_precision(tp, fp)
      DC_segmentation += segmentation_dice_coefficient(tp, fn, fp)
      
      meter['loss'] = avg_loss_train
      meter['acc_detection'] = acc_detection
      meter['recall_detection'] = recall_detection
      meter['precision_detection'] = precision_detection
      meter['DC_detection'] = DC_detection
      meter['acc_segmentation'] = acc_segmentation
      meter['recall_segmentation'] = recall_segmentation
      meter['precision_segmentation'] = precision_segmentation
      meter['DC_segmentation'] = DC_segmentation
      
      for k, val in meter.items():
        avg_meter[k] = val
        pbar.set_postfix(**{k: "{:.5f}".format(v / (length)) for k, v in avg_meter.items()})
    
    avg_loss_train = avg_loss_train/iterations
    
    acc_detection = acc_detection/iterations
    recall_detection = recall_detection/iterations
    precision_detection = precision_detection/iterations
    DC_detection = DC_detection/iterations
    
    acc_segmentation = acc_segmentation/iterations
    recall_segmentation = recall_segmentation/iterations
    precision_segmentation = precision_segmentation/iterations
    DC_segmentation = DC_segmentation/iterations
    
    # Print the evaluation results
    print('Epoch [%d/%d], Loss: %.4f, \n[Training], DiceS: %.4f, DiceD: %.4f, AccuracyS: %.4f, AccuracyD: %.4f, RecallS: %.4f, RecallD: %.4f, PrecisionS: %.4f, PrecisionD: %.4f' % (
      epoch+1, self.num_epochs, \
      avg_loss_train, \
      DC_segmentation, \
      DC_detection, \
      acc_segmentation, \
      acc_detection, \
      recall_segmentation, \
      recall_detection, \
      precision_segmentation, \
      precision_detection, \
      ))
    
    writer_train_error.add_scalar('Train LOSS', avg_loss_train, iterations)
    writer_train_acc.add_scalar('Train Accuracy', acc_detection, iterations)
    writer_train_dice.add_scalar('Train Dice', DC_detection, iterations)
    
    checkpoint = {'model_state_dict': self.attUnet.state_dict(),
                  'optimizer_state_dict': self.optimizer.state_dict(),
                  'loss': avg_loss_train,
                  'epoch': epoch}
    
    torch.save(checkpoint, '/home/user3/myenv/checkpoints/checkpoint.pt')    

  def per_epoch_valid(self,epoch):
    self.attUnet.train(False)
    self.attUnet.eval()
    
    acc_detection = 0.
    recall_detection = 0.
    precision_detection = 0.
    DC_detection = 0.
    
    acc_segmentation = 0.
    recall_segmentation = 0.
    precision_segmentation = 0.
    DC_segmentation = 0.
    
    avg_loss_valid = 0.
    length = 0
    iterations = 0
    
    for i, (images, GT) in enumerate(self.valid_loader):
      
      images = images.to(self.device)
      GT = GT.to(self.device)
      
      with torch.no_grad():
        SR = self.attUnet(images)
      
      length += images.size(0)
      iterations += 1
      
      #criterion = MixedLoss(alpha=0.25, gamma=2)
      #loss = criterion(SR_probs, GT)
      
      #loss = bce_dice_loss(SR_probs, GT)
           
      loss = weighted_bce(SR, GT)
      avg_loss_valid += loss.item()
        
      SR_probs = torch.nn.functional.sigmoid(SR)
      SR_numpy = (np.squeeze(SR).data).cpu().numpy()
      GT_numpy = (np.squeeze(GT).data).cpu().numpy()
      
      tpd, tnd, fpd, fnd = detection_rate(SR_numpy, GT_numpy)
      tp, tn, fp, fn = segmentation_rate(SR_numpy, GT_numpy)
      
      acc_detection += detection_accuracy(tpd, tnd, fpd, fnd)
      recall_detection += detection_recall(tpd, fnd)
      precision_detection += detection_precision(tpd, fpd)
      DC_detection += detection_dice_coefficient(tpd, fnd, fpd)
      acc_segmentation += segmentation_accuracy(tp, tn, fp, fn)
      recall_segmentation += segmentation_recall(tp, fn)
      precision_segmentation += segmentation_precision(tp, fp)
      DC_segmentation += segmentation_dice_coefficient(tp, fn, fp)
      
    avg_loss_valid = avg_loss_valid/iterations
    
    acc_detection = acc_detection/iterations
    recall_detection = recall_detection/iterations
    precision_detection = precision_detection/iterations
    DC_detection = DC_detection/iterations
    
    acc_segmentation = acc_segmentation/iterations
    recall_segmentation = recall_segmentation/iterations
    precision_segmentation = precision_segmentation/iterations
    DC_segmentation = DC_segmentation/iterations
    
    print('Epoch [%d/%d], Loss: %.4f, \n[Validation], DiceS: %.4f, DiceD: %.4f, AccuracyS: %.4f, AccuracyD: %.4f, RecallS: %.4f, RecallD: %.4f, PrecisionS: %.4f, PrecisionD: %.4f' % (
      epoch+1, self.num_epochs, \
      avg_loss_valid, \
      DC_segmentation, \
      DC_detection, \
      acc_segmentation, \
      acc_detection, \
      recall_segmentation, \
      recall_detection, \
      precision_segmentation, \
      precision_detection, \
      ))
    
    writer_valid_error.add_scalar('Valid LOSS', avg_loss_valid, iterations)
    writer_valid_acc.add_scalar('Valid Accuracy', acc_detection, iterations)
    writer_valid_dice.add_scalar('Valid Dice', DC_detection, iterations)
    
    #self.early_stopping(avg_loss_valid, self.attUnet)

    #if self.early_stopping.early_stop:
      #print("Early stopping")
      #raise EarlyStopError

    ### Save model to a new file
    self.model_path = '/home/user3/myenv/models'
    attUnet_path = os.path.join(self.model_path, 'Model: %s-%d-%.4f-%d.pkl' % (self.model_type, self.num_epochs, self.lr, self.batch_size))
    torch.save(self.attUnet.state_dict(), attUnet_path)
  

In [0]:
model = Solver(train_loader, valid_loader, test_loader)

In [0]:
model.train()

In [0]:
### Evaluation for one batch

image, gt = next(iter(test_loader));


imageTS = image

image = image.permute(0,2,3,1).numpy()
image = image*np.array((0.5, 0.5, 0.5)) + np.array((0.5, 0.5, 0.5))

### Plot images and groud truth
plot_images(image*255, gt.numpy()*255, cls_pred=None, smooth=True)

### Pass image to the model

model.attUnet.train(False)
model.attUnet.eval()

imageTS = imageTS.to(model.device)

with torch.no_grad():
    SR = model.attUnet(imageTS)

SR_probs = torch.nn.functional.sigmoid(SR)

temp = np.squeeze(SR_probs.permute(0,2,3,1)).detach().cpu().numpy()


In [0]:
print('Evaluation per image')

tpd, tnd, fpd, fnd = detection_rate(temp, gt)
tp, tn, fp, fn = segmentation_rate(temp, gt)
"""
print('\n Segmentation results')
acc_segmentation = segmentation_accuracy(tp, tn, fp, fn)
print(acc_segmentation)
recall_segmentation = segmentation_recall(tp, fn)
print(recall_segmentation)
precision_segmentation = segmentation_precision(tp, fp)
print(precision_segmentation)
DC_segmentation = segmentation_dice_coefficient(tp, fn, fp)
print(DC_segmentation)
"""
print('\nDetection results')
acc_detection = detection_accuracy(tpd, tnd, fpd, fnd)
print("Acc: {}".format(acc_detection))
recall_detection = detection_recall(tpd, fnd)
print("Recall: {}".format(recall_detection))
precision_detection = detection_precision(tpd, fpd)
print("Precision: {}".format(precision_detection))
DC_detection = detection_dice_coefficient(tpd, fnd, fpd)
print("Dice: {}".format(DC_detection))

print('\nDetection rates')
print("True Positive: {}".format(tpd))
print("True Negative: {}".format(tnd))
print("False Positive: {}".format(fpd))
print("False Negative: {}".format(fnd))

"""
print('\n Segmentation rates')
print(tp)
print(tn)
print(fp)
print(fn)
"""

In [0]:
plot_images(image*255, temp*255, cls_pred=None, smooth=True)

In [0]:
#-----------------Testing-----------------#

model.attUnet.train(False)
model.attUnet.eval()

#attUnet_path = os.path.join(self.model_path, 'Net-100-0.0000-0-0.5000.pkl')
#self.attUnet.load_state_dict(torch.load(attUnet_path))

length = 0
meter = {}

acc_detection = 0.
recall_detection = 0.
precision_detection = 0.
DC_detection = 0.
acc_segmentation = 0.
recall_segmentation = 0.
precision_segmentation = 0.
DC_segmentation = 0.
F1_detection = 0.
F2_detection = 0.
tpd_=[]
tnd_=[]
fpd_=[]
fnd_=[]
tp_=[]
tn_=[]
fp_=[]
fn_=[]
TP_number = 0
FP_number = 0
FN_number = 0

for  i, (images, GT) in enumerate(test_loader):

    images = images.to(model.device)
    
    with torch.no_grad():
        SR = model.attUnet(images)
        
    SR_probs = torch.nn.functional.sigmoid(SR)
    
    length += images.size(0)

    SR_numpy = (np.squeeze(SR_probs).data).cpu().numpy()
    GT_numpy = (np.squeeze(GT).data).cpu().numpy()

    tpd, tnd, fpd, fnd = detection_rate(SR_numpy, GT_numpy)
    tp, tn, fp, fn = segmentation_rate(SR_numpy, GT_numpy)

    tpd_.append(tpd) 
    tnd_.append(tnd)
    fpd_.append(fpd)
    fnd_.append(fnd)
    tp_.append(tp)
    tn_.append(tn)
    fp_.append(fp)
    fn_.append(fn)

acc_detection = detection_accuracy(sum(tpd_), sum(tnd_), sum(fpd_), sum(fnd_))
recall_detection = detection_recall(sum(tpd_), sum(fnd_))
precision_detection = detection_precision(sum(tpd_), sum(fpd_))
DC_detection = detection_dice_coefficient(sum(tpd_), sum(fnd_), sum(fpd_))

acc_segmentation = segmentation_accuracy(sum(tp_), sum(tn_), sum(fp_), sum(fn_))
recall_segmentation = segmentation_recall(sum(tp_), sum(fn_))
precision_segmentation = segmentation_precision(sum(tp_), sum(fp_))
DC_segmentation = segmentation_dice_coefficient(sum(tp_), sum(fn_), sum(fp_))

F1_detection = (2*precision_detection*recall_detection)/(precision_detection+recall_detection)
F2_detection = (5*precision_detection*recall_detection)/(4*precision_detection+recall_detection) 

TP_number = sum(tpd_)
TN_number = sum(tnd_)
FP_number = sum(fpd_)
FN_number = sum(fnd_)

# Print the evaluation results
print('[Testing], DiceS: %.4f, DiceD: %.4f, AccuracyS: %.4f, AccuracyD: %.4f, RecallS: %.4f, RecallD: %.4f, PrecisionS: %.4f, PrecisionD: %.4f, F1_detection: %.4f, F2_detection: %.4f TP_Number: %.4f, TN_Number: %.4f, FP_Number: %.4f, FN_Number: %.4f' % (
    DC_segmentation, \
    DC_detection, \
    acc_segmentation, \
    acc_detection, \
    recall_segmentation, \
    recall_detection, \
    precision_segmentation, \
    precision_detection, \
    F1_detection, \
    F2_detection, \
    TP_number, \
    TN_number, \
    FP_number, \
    FN_number, \
    ))

writer_test_acc.add_scalar('Detection Accuracy', acc_detection)


In [0]:
    
    encoder_params = list(filter(lambda kv: kv[0] in my_list, self.attUnet.named_parameters()))
    
    self.optimizer = optim.Adam([
    {"params": encoder_params, "lr": 0.000001},
    { "params" : self.attUnet.module.conv0_1.conv1.weight, "lr": 0.00001},
    { "params" : self.attUnet.module.conv0_1.conv1.bias, "lr": 0.00001},
    { "params" : self.attUnet.module.conv0_1.bn1.weight, "lr": 0.00001},
    { "params" : self.attUnet.module.conv0_1.bn1.bias, "lr": 0.00001},
    { "params" : self.attUnet.module.conv0_1.conv2.weight, "lr": 0.00001},
    { "params" : self.attUnet.module.conv0_1.conv2.bias, "lr": 0.00001},
    { "params" : self.attUnet.module.conv0_1.bn2.weight, "lr": 0.00001},
    { "params" : self.attUnet.module.conv0_1.bn2.bias, "lr": 0.00001},
    { "params" : self.attUnet.module.conv0_2.conv1.weight, "lr": 0.00001},
    { "params" : self.attUnet.module.conv0_2.conv1.bias, "lr": 0.00001},
    { "params" : self.attUnet.module.conv0_2.bn1.weight, "lr": 0.00001},
    { "params" : self.attUnet.module.conv0_2.bn1.bias, "lr": 0.00001},
    { "params" : self.attUnet.module.conv0_2.conv2.weight, "lr": 0.00001},
    { "params" : self.attUnet.module.conv0_2.conv2.bias, "lr": 0.00001},
    { "params" : self.attUnet.module.conv0_2.bn2.weight, "lr": 0.00001},
    { "params" : self.attUnet.module.conv0_2.bn2.bias, "lr": 0.00001},
    { "params" : self.attUnet.module.conv0_3.conv1.weight, "lr": 0.00001},
    { "params" : self.attUnet.module.conv0_3.conv1.bias, "lr": 0.00001},
    { "params" : self.attUnet.module.conv0_3.bn1.weight, "lr": 0.00001},
    { "params" : self.attUnet.module.conv0_3.bn1.bias, "lr": 0.00001},
    { "params" : self.attUnet.module.conv0_3.conv2.weight, "lr": 0.00001},
    { "params" : self.attUnet.module.conv0_3.conv2.bias, "lr": 0.00001},
    { "params" : self.attUnet.module.conv0_3.bn2.weight, "lr": 0.00001},
    { "params" : self.attUnet.module.conv0_3.bn2.bias, "lr": 0.00001},
    { "params" : self.attUnet.module.conv1_1.conv1.weight, "lr": 0.00001},
    { "params" : self.attUnet.module.conv1_1.conv1.bias, "lr": 0.00001},
    { "params" : self.attUnet.module.conv1_1.bn1.weight, "lr": 0.00001},
    { "params" : self.attUnet.module.conv1_1.bn1.bias, "lr": 0.00001},
    { "params" : self.attUnet.module.conv1_1.conv2.weight, "lr": 0.00001},
    { "params" : self.attUnet.module.conv1_1.conv2.bias, "lr": 0.00001},
    { "params" : self.attUnet.module.conv1_1.bn2.weight, "lr": 0.00001},
    { "params" : self.attUnet.module.conv1_1.bn2.bias, "lr": 0.00001},
    { "params" : self.attUnet.module.conv1_2.conv1.weight, "lr": 0.00001},
    { "params" : self.attUnet.module.conv1_2.conv1.bias, "lr": 0.00001},
    { "params" : self.attUnet.module.conv1_2.bn1.weight, "lr": 0.00001},
    { "params" : self.attUnet.module.conv1_2.bn1.bias, "lr": 0.00001},
    { "params" : self.attUnet.module.conv1_2.conv2.weight, "lr": 0.00001},
    { "params" : self.attUnet.module.conv1_2.conv2.bias, "lr": 0.00001},
    { "params" : self.attUnet.module.conv1_2.bn2.weight, "lr": 0.00001},
    { "params" : self.attUnet.module.conv1_2.bn2.bias, "lr": 0.00001},
    { "params" : self.attUnet.module.conv2_1.conv1.weight, "lr": 0.00001},
    { "params" : self.attUnet.module.conv2_1.conv1.bias, "lr": 0.00001},
    { "params" : self.attUnet.module.conv2_1.bn1.weight, "lr": 0.00001},
    { "params" : self.attUnet.module.conv2_1.bn1.bias, "lr": 0.00001},
    { "params" : self.attUnet.module.conv2_1.conv2.weight, "lr": 0.00001},
    { "params" : self.attUnet.module.conv2_1.conv2.bias, "lr": 0.00001},
    { "params" : self.attUnet.module.conv2_1.bn2.weight, "lr": 0.00001},
    { "params" : self.attUnet.module.conv2_1.bn2.bias, "lr": 0.00001},
    { "params" : self.attUnet.module.conv1.weight, "lr": 0.00001},
    { "params" : self.attUnet.module.bn1.weight, "lr": 0.00001},
    { "params" : self.attUnet.module.bn1.bias, "lr": 0.00001},
    { "params" : self.attUnet.module.conv2.weight, "lr": 0.00001},
    { "params" : self.attUnet.module.bn2.weight, "lr": 0.00001},
    { "params" : self.attUnet.module.bn2.bias, "lr": 0.00001},
    { "params" : self.attUnet.module.conv3.weight, "lr": 0.00001},
    { "params" : self.attUnet.module.bn3.weight, "lr": 0.00001},
    { "params" : self.attUnet.module.bn3.bias, "lr": 0.00001},
    { "params" : self.attUnet.module.conv4.weight, "lr": 0.00001},
    { "params" : self.attUnet.module.bn4.weight, "lr": 0.00001},
    { "params" : self.attUnet.module.bn4.bias, "lr": 0.00001},
    { "params" : self.attUnet.module.dupsample4.conv_w.weight, "lr": 0.00001},
    { "params" : self.attUnet.module.dupsample4.conv_p.weight, "lr": 0.00001},
    { "params" : self.attUnet.module.conv6.weight, "lr": 0.00001},
    { "params" : self.attUnet.module.bn6.weight, "lr": 0.00001},
    { "params" : self.attUnet.module.bn6.bias, "lr": 0.00001},
    { "params" : self.attUnet.module.final.weight, "lr": 0.00001},
    { "params" : self.attUnet.module.final.bias, "lr": 0.0001}], self.lr, [self.beta1, self.beta2])

    encoder_params = list(filter(lambda kv: kv[0] in my_list, self.attUnet.named_parameters()))
    
    self.optimizer = optim.Adam([
    {"params": encoder_params, "lr": 0.000001},
    { "params" : self.attUnet.module.conv0_1.conv1.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv0_1.conv1.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv0_1.bn1.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv0_1.bn1.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv0_1.conv2.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv0_1.conv2.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv0_1.bn2.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv0_1.bn2.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv0_2.conv1.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv0_2.conv1.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv0_2.bn1.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv0_2.bn1.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv0_2.conv2.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv0_2.conv2.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv0_2.bn2.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv0_2.bn2.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv0_3.conv1.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv0_3.conv1.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv0_3.bn1.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv0_3.bn1.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv0_3.conv2.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv0_3.conv2.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv0_3.bn2.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv0_3.bn2.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv1_1.conv1.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv1_1.conv1.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv1_1.bn1.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv1_1.bn1.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv1_1.conv2.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv1_1.conv2.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv1_1.bn2.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv1_1.bn2.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv1_2.conv1.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv1_2.conv1.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv1_2.bn1.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv1_2.bn1.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv1_2.conv2.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv1_2.conv2.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv1_2.bn2.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv1_2.bn2.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv1_3.conv1.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv1_3.conv1.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv1_3.bn1.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv1_3.bn1.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv1_3.conv2.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv1_3.conv2.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv1_3.bn2.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv1_3.bn2.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv2_1.conv1.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv2_1.conv1.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv2_1.bn1.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv2_1.bn1.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv2_1.conv2.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv2_1.conv2.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv2_1.bn2.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv2_1.bn2.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv2_2.conv1.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv2_2.conv1.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv2_2.bn1.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv2_2.bn1.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv2_2.conv2.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv2_2.conv2.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv2_2.bn2.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv2_2.bn2.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv3_1.conv1.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv3_1.conv1.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv3_1.bn1.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv3_1.bn1.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv3_1.conv2.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv3_1.conv2.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv3_1.bn2.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv3_1.bn2.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv0_4.conv1.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv0_4.conv1.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv0_4.bn1.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv0_4.bn1.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv0_4.conv2.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv0_4.conv2.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.conv0_4.bn2.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.conv0_4.bn2.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.final.weight, "lr": 0.0001},
    { "params" : self.attUnet.module.final.bias, "lr": 0.0001},
    { "params" : self.attUnet.module.final.bias, "lr": 0.0001}], self.lr, [self.beta1, self.beta2])

The number of parameters: 96001153

